# What?

**Stand: ** 27/06/27

Mittels 
```bash
$ python 02a_mark_moved_frames_manual.py
```
wurde pro Tag festgestellt ob sich die Kamera bewegt hat.

**Toc:**
1. [Übersicht](#Übersicht)
2. [gTimeline](#Vorbereitung-f%C3%BCr-das-Darstellen-mittels-gTimeline)

In [1]:
%matplotlib inline
import datetime
import json

import iso8601
import pandas as pd

import modules.pd_utils as pd_utils
import modules.gtimeline_wrapper as gtwrapper

In [2]:
def get_proxy_day_str(row, column):
    """Retruns the day of the timestamp in `column`"""
    return row[column].strftime("%Y-%m-%d")

def get_proxy_day_start(row, column):
    """Retruns the day of the timestamp in `column`"""
    return iso8601.parse_date(row[column])

def get_proxy_day_end(row, column):
    """Retruns the day of the timestamp in `column`"""
    return iso8601.parse_date(row[column]) + datetime.timedelta(0, 86340)

In [3]:
path_0 = 'derived_data/02_man_marked_moved_frames/manuel_Cam_0.csv'
path_1 = 'derived_data/02_man_marked_moved_frames/manuel_Cam_1.csv'
path_2 = 'derived_data/02_man_marked_moved_frames/manuel_Cam_2.csv'
path_3 = 'derived_data/02_man_marked_moved_frames/manuel_Cam_3.csv'
paths=[path_0, path_1, path_2, path_3]

In [4]:
all_data = []
for path in paths:
    df = pd.read_csv(path, sep=';')
    df_cam_ts_fst = df.apply(lambda row: pd_utils.extract_metadata_videoname(row['first_frame']), axis = 1)
    df_cam_ts_sec = df.apply(lambda row: pd_utils.extract_metadata_videoname(row['second_frame']), axis = 1)

    df_cam_ts_fst.columns = ['fst_cam_id', 'fst_ts_start', 'fst_ts_end']
    df_cam_ts_sec.columns = ['sec_cam_id', 'sec_ts_start', 'sec_ts_end']

    df = df.merge(df_cam_ts_fst, left_index=True, right_index=True)
    df = df.merge(df_cam_ts_sec, left_index=True, right_index=True)

    df['proxy_day'] = df.apply(lambda row: get_proxy_day_str(row, 'fst_ts_end'), axis = 1)
    df['proxy_day_start'] = df.apply(lambda row: get_proxy_day_start(row, 'proxy_day'), axis = 1)
    df['proxy_day_end'] = df.apply(lambda row: get_proxy_day_end(row, 'proxy_day'), axis = 1 )
    all_data.append(df)

Die Frames stammen immer aus dem ersten Frame eines Videos, welches um die Uhrzeit 0 aufgenommen wurde. Eine Video wie

`Cam_0_2016-07-20T23:55:06.659237Z--2016-07-21T00:00:54.807835Z.jpg`

wird als Stellvertreter für den Tag `2016-07-21` betrachtet.

# Übersicht

In [19]:
disp_list = []
for df in all_data:
    df = df[['proxy_day', 'moved', 'fst_cam_id']]
    disp_list.append(df)

In [20]:
df = pd.merge(disp_list[0], disp_list[1],suffixes=('', '_cam_1'),how='outer', on='proxy_day')
df = pd.merge(df, disp_list[2],suffixes=('', '_cam_2'),how='outer', on='proxy_day')
df = pd.merge(df, disp_list[3],suffixes=('', '_cam_3'),how='outer', on='proxy_day')

In [21]:
df = df.rename(columns={'moved':'moved_cam_0'})

In [22]:
df = df[['proxy_day','moved_cam_0','moved_cam_1', 'moved_cam_2', 'moved_cam_3']]

In [24]:
def color_movement_red(val):
    if pd.isnull(val):
        color = 'yellow'
    elif val:
        color = 'red'
    else:
        color = 'white'
    # color = 'red' if val else 'white'
    return 'background-color: %s' % color

df.style.applymap(color_movement_red, subset=['moved_cam_0','moved_cam_1', 'moved_cam_2', 'moved_cam_3'])

In [26]:
df.to_pickle('./derived_data/02_man_marked_moved_frames/02b_df_man_marked_list.pkl')

# Vorbereitung für das Darstellen mittels gTimeline

In [10]:
df = pd.concat(all_data, ignore_index=True)

def adjust_moved(row, col):
    if row[col] == 1:
        ret = 'moved'
    else:
        ret = 'stable'
    return ret

df['moved'] = df.apply(lambda row: adjust_moved(row, 'moved'), axis = 1)

In [11]:
def add_tooltip(row):
    string="""
    <table style="font-size:10px">
        <tr>
            <td><b>Moved:</b></td><td>{moved}</td>
        </tr>
        <tr>
            <td><b>Day:</b></td> <td>{day}</td>
        </tr>
        <tr>
            <td><b>1. Reference Image:</b></td> <td>{fst_frame}</td>
        </tr>
        <tr>
            <td><b>2. Reference Image:</b></td> <td>{sec_frame}</td>
        </tr>
    </table>
    """.format(moved=row['moved'],
               day=row['proxy_day_start'].strftime('%Y-%m-%d'),
               fst_frame=row['first_frame'],
               sec_frame=row['second_frame'])
    return string

In [12]:
df['tooltip'] = df.apply(add_tooltip, axis = 1)
df['row_label'] = df.apply(lambda row: 'Cam_{id} manual marked'.format(id=row['fst_cam_id']), axis=1)

In [13]:
export_df = df.copy()
export_df = export_df[['row_label','proxy_day_start', 'proxy_day_end','moved', 'tooltip']]
export_df.columns=['row_label', 'start', 'end', 'bar_label', 'tooltip']

In [14]:
export_df.to_pickle('./derived_data/02_man_marked_moved_frames/df_man_marked.pkl')

In [15]:
g_dict = gtwrapper.create_gtimeline_dict(export_df, 'row_label', 'bar_label', 'start', 'end', 'tooltip')

with open('docs/02_mark_moved_frames_manual/data.json', 'w') as fp:
    json.dump(g_dict, fp)

In [16]:
!firefox -p develop docs/02_mark_moved_frames_manual/02_mark_moved_frames_manual.html

In [17]:
!jupyter nbconvert --to notebook --execute 04_overall_anaylses.ipynb

[NbConvertApp] Converting notebook 04_overall_anaylses.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 1906 bytes to 04_overall_anaylses.nbconvert.ipynb
